In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

BLOCK_SIZE = 8
BATCH_SIZE = 4

cuda


In [12]:
with open('wizard of oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
80


In [13]:
#Character level tokeniser

string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


In [14]:
data = torch.tensor(encode(text), dtype=torch.long) #pytorch tensor
print(data[:100]) #print first 100 encoded chars

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


Bigram start of content -> h -> e -> l -> l -> o

Given previous char, we predit the incomming one
We take a block_size ( block of characters) shaped tensor and predict the next token after this block.



In [15]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [16]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    print('ix:', ix)
    x = torch.stack([data[i:i + BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i + 1: i + BLOCK_SIZE] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y 

x, y = get_batch('train')
print('inputs:\n', x)
print('targets:\n', y)

    

ix: tensor([ 24837, 107538,   7292,  45158])
inputs:
 tensor([[11,  0,  0,  3, 39, 59,  1, 56],
        [57,  1, 76, 54, 72,  1, 54, 67],
        [68, 76, 67,  1, 54, 65, 66, 68],
        [ 1, 61, 62, 72,  1, 58, 54, 71]], device='cuda:0')
targets:
 tensor([[ 0,  0,  3, 39, 59,  1, 56],
        [ 1, 76, 54, 72,  1, 54, 67],
        [76, 67,  1, 54, 65, 66, 68],
        [61, 62, 72,  1, 58, 54, 71]], device='cuda:0')


In [17]:
x = train_data[:BLOCK_SIZE]
y = train_data[1: BLOCK_SIZE + 1]

#Sequential way
for t in range(BLOCK_SIZE):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target) 

when input is tensor([1]) target is tensor(1)
when input is tensor([1, 1]) target is tensor(28)
when input is tensor([ 1,  1, 28]) target is tensor(39)
when input is tensor([ 1,  1, 28, 39]) target is tensor(42)
when input is tensor([ 1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([ 1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([ 1,  1, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([ 1,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


We want to process theses sequences in parallel with GPU, using mini-batches

In [25]:
class BigramLanguageModel(nn.Module): 
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) # Probability distrib of the next token

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape #Batches, Time, Channel = nb of classes
    
            logits = logits.view(B*T, C)
            target = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss # Logits = unnormalized score of the LLM for each token incomming

    def generate(self, index, max_new_tokens):
        # Index is (B, T) array of indices int the current context
        for _ in range(max_new_tokens):
            # Get the preditions
            logits, loss = self.forward(index)
            # Focus only on the last time step
            logits = logits[:, -1, :] # Becomes (B, C)
    
            # Apply softmax to get the probabilities
            proba = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution
            index_next = torch.multinomial(proba, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=-1) #(B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


86DP9wPJH9SO!EhgI!u,Q1k3*'d0g&KA66Y_4bbH:MUnn21[)zhX,_uiw19-65;
]H9"B74:at00Qn9OY'g
lEGv)u,-Iq"om,UJ
P9W;kgg[)328efDzhojG9'w.OGvYWTxkd9UM*btQN 3u,Qdg*]cAkrht
80rZ6pD*'ucC!ds!Xh_C;zbNtSNVd1TF8TsTO_wn3yPQIC!zX2f
ZzhfR0mx5("o,EcLAuX)bzajYSB*L("oeG:k;5JLeSqTS'atF9b_W"h*U:CV-5t0101ADKMB'UkGeZE3u5JLT *ZnSH. 4)]FUDKQCfZ"z h2RwXK?1T,.sH]!Ehsids2H7&OM*Xim8NV_ALTxRSoU_( QDNZNr].00'V&&5Jg-GpefZ6SY''Z:'u8YRrXmp?
lNcATO](Qtx[VS[atHHHT'6v_[TIauP6ab;w?jGelQ3RXeh6a-feW144BjxI?sLNwHTZ9N(-ZtG,EuCw4mp8nj,HoB0JL'dA
